# Tutorial 1. Controlling a basic experiment using MeasurementControl 

This tutorial covers basic usage of quantify focusing on running basic experiments using `MeasurementControl`. 
The `MeasurementControl` is the main `Instrument` in charge of running any experiment. It takes care of saving the data in a standardized format as well as live plotting of the data during the experiment. 
`quantify` makes a distinction between `soft`(ware) controlled measurements and `hard`(ware) controlled measurements. 

In a `soft` measurement `MeasurementControl` is in charge of the measurement loop and consecutively sets and gets datapoints. A `soft` measurement can be 1D, 2D or higher dimensional and also supports (TODO) adaptive measurements in which the datapoints are determined during the measurement loop. 

In a `hard` measurement the hardware (such as an AWG or a central controller) is in charge of the measurement loop. In this case, the datapoints to be acquired are determined before the experiment starts and are precompiled into the hardware which is then armed and starts acquisition. In a `hard` measurement `MeasurementControl` does not take care of the measurement loop but still takes care of the data storage and live plotting of the experiment. 

This tutorial is structured as follows. 
In the first section we use a 1D `soft`(ware) controlled loop to explain the flow of a basic experiment. We start by setting up a noisy cosine model to serve as our mock setup and then use the `MC` to measure this. We then perform basic (manual) analysis on the data from this experiment. We show how to find and load a dataset, perform a basic fit, and store the results. 
In the later tutorials we show more advanced usage of the Measurement control including `hard`(ware) controlled experiments (todo) and adaptive loops (todo).


In [ ]:
import numpy as np 
import xarray as xr 
import matplotlib.pyplot as plt 
from qcodes import ManualParameter, Parameter
%matplotlib inline

In [ ]:
from quantify.measurement import MeasurementControl
from quantify.measurement.core import Settable, Gettable
import quantify.visualization.pyqt_plotmon as pqm

In [ ]:
MC = MeasurementControl('MC')
plotmon = pqm.PlotMonitor_pyqt('plotmon')
MC.instr_plotmon(plotmon.name)

In [ ]:
MC.instr_plotmon.get_instr().tuid()

## A 1D soft(ware) controlled loop

### Define a simple model 
We start by defining a simple model to mock our experiment setup. We will be generating a cosine with some normally distributed noise added on top of it. 

In [ ]:
from time import sleep

In [ ]:
def cos_func(t, amplitude, frequency, phase, offset):
    """A simple cosine function"""
    return amplitude * np.cos(2 * np.pi * frequency * t + phase) + offset

# Parameters are created to emulate a system being measured 
amp = ManualParameter('amp', initial_value=1, unit='V', label='Amplitude')
freq = ManualParameter('freq', initial_value=.5, unit='Hz', label='Frequency')
t = ManualParameter('t', initial_value=1, unit='s', label='Time')
phi = ManualParameter('phi', initial_value=0, unit='Rad', label='Phase')

# we add in some noise to make the fitting example later on more interesting
noise_level = ManualParameter('noise_level', initial_value=0.05, unit='V', label='Noise level')

acq_delay = ManualParameter('acq_delay', initial_value=.1, unit='s')

def cosine_model():
    sleep(acq_delay())
    return cos_func(t(), amp(), freq(), phase=phi(), offset=0) +np.random.randn()*noise_level()

# We wrap our function in a Parameter to be able to give 
sig = Parameter(name='sig', label='Signal level', unit='V', get_cmd=cosine_model)

In [ ]:
acq_delay(0.0) # by setting this to a non-zero value we can see the live plotting in action. 

### Running the experiment

The complete experiment is defined in just 4 lines of code. We specify what parameter we want to set, time `t` in this case, what points to measure at, and what parameter to measure. 
We then tell the MeasurementControl `MC` to run which will return an `xarray:Dataset` object. 

We use the `Settable` and `Gettable` helper classes to ensure our parameters contain the correct attributes.

In [ ]:
MC.set_setpars(Settable(t))
MC.set_setpoints(np.linspace(0, 5, 50))
MC.set_getpars(Gettable(sig))
dset = MC.run('Cosine test')


In [ ]:
plotmon.main_QtPlot

In [ ]:
dset.attrs['tuid']

In [ ]:
dset


We can play with some live plotting options to see how the MC behaves when changing the update interval. 

In [ ]:
# By default the MC updates the datafile and live plot every 0.1 seconds to reduce overhead. 
# the total overhead is ~0.05s per update
MC.update_interval(0.01) # Setting it even to 0.01 makes a dramatic difference, try it out!

In [ ]:
MC.set_setpars(Settable(t))
MC.set_setpoints(np.linspace(0, 50, 1000))
MC.set_getpars(Gettable(sig))
dset = MC.run('Many points live plot test')

In [ ]:
noise_level(0) #let's disable noise from here on to get prettier figures

### Analyzing the experiment

#### Loading the data
The `Dataset` contains all the information required to perform basic analysis of the experiment and information on where the data is stored. 
We can alternatively load the dataset from disk based on it's `tuid`, a timestamp-based unique identifier. If you do not know the tuid of the experiment you can find the latest tuid containing a certain string in the experiment name using `quantify.data_handling.get_latest_tuid`. See the data storage documentation for more details on the folder structure and files contained in the data directory. 



In [ ]:
from quantify.data.data_handling import load_dataset, get_latest_tuid

# here we look for the latest datafile in the datadirectory named "Cosine test"
# note that this is not he last dataset but one dataset earlier
tuid = get_latest_tuid('Cosine test') 
print('tuid: {}'.format(tuid))
dset = load_dataset(tuid)

dset

#### Performing fits and extracting quantities of interest

We have used a cosine function "mock" an experiment, the goal of the experiment is to find the underlying parameters. 
We extract these parameters by performing a fit to a model, which coincidently, is based on the same cosine function. 
For fitting we recommend using the lmfit library.  See https://lmfit.github.io/lmfit-py/model.html on how to fit data to a custom model.

In [ ]:
import lmfit
# we create a model based on our function
mod = lmfit.Model(cos_func)
# and specify initial guesses for each parameter
mod.set_param_hint('amplitude', value=.8, vary=True)
mod.set_param_hint('frequency', value=.4)
mod.set_param_hint('phase', value=0, vary=False)
mod.set_param_hint('offset', value=0, vary=False)
params = mod.make_params()
# and here we perform the fit. 
fit_res = mod.fit(dset['y0'].values, t=dset['x0'].values, params=params)


# It is possible to get a quick visualization of our fit using a build-in method of lmfit
fit_res.plot_fit(show_init=True)

In [ ]:
fit_res.params

In [ ]:
# And we can print an overview of the fitting results
print(fit_res.fit_report())

#### Plotting and saving the results of the analysis

In [ ]:
# We include some visualization utilities in quantify
from quantify.visualization.SI_utilities import set_xlabel, set_ylabel

In [ ]:
f, ax =plt.subplots()

ax.plot(dset['x0'], dset['y0'], marker='o', label='Data')
x_fit = np.linspace(dset['x0'][0], dset['x0'][-1], 1000)
y_fit = cos_func(t=x_fit, **fit_res.best_values)
ax.plot(x_fit, y_fit, label='Fit')
ax.legend()

set_xlabel(ax, dset['x0'].attrs['long_name'], dset['x0'].attrs['unit'])
set_ylabel(ax, dset['y0'].attrs['long_name'], dset['y0'].attrs['unit'])
ax.set_title('{}\n{}'.format(tuid, 'Cosine test'))

Now that we have analyzed our data and created a figure, we probably want to store the results of our analysis. 
We will want to store the figure and the results of the fit in the `experiment folder`. 


In [ ]:
from os.path import join
from quantify.data.data_handling import create_exp_folder
exp_folder = create_exp_folder(dset.tuid, dset.name)

In [ ]:
lmfit.model.save_modelresult(fit_res, join(exp_folder, 'fit_res.json'))
f.savefig(join(exp_folder, 'Cosine fit.png'), dpi=300, bbox_inches='tight')

## A 2D soft(ware) controlled loop

It is often desired to measure heatmaps (2D grids) of some parameter. 
This can be done by specifying two set parameters. 
The setpoints of the grid can be specified in two ways. 

### Method 1, a quick grid. 

In [ ]:
MC.update_interval(.1)

In [ ]:
times = np.linspace(0, 5, 500)
amps = np.linspace(-1, 1, 31)

MC.set_setpars([Settable(t), Settable(amp)])
MC.set_setpoints_grid([times, amps])
MC.set_getpars(Gettable(sig))
dset = MC.run('2D Cosine test')

In [ ]:
plotmon.main_QtPlot

In [ ]:
plotmon.secondary_QtPlot

### Method 2, custom tuples in 2D 

N.B. it is also possible to do this for higher dimensional loops

In [ ]:
r = np.linspace(0, 1.5, 2000)
dt = np.linspace(0, 1, 2000)

f = 10

theta = np.cos(2*np.pi*f*dt)
def polar_coords(r, theta):

    x = r*np.cos(2*np.pi*theta)
    y = r*np.sin(2*np.pi*theta)
    return x, y 

x,y = polar_coords(r, theta)
setpoints = np.column_stack([x,y])

In [ ]:
acq_delay(0.001)
MC.update_interval(0.5)

In [ ]:
MC.set_setpars([t, amp])
MC.set_setpoints(setpoints)
MC.set_getpars(sig)
dset = MC.run('2D radial setpoints')

In [ ]:
plotmon.main_QtPlot

In [ ]:
plotmon.secondary_QtPlot

### Below is for testing

In [ ]:
# This snippet exists to quickly reload and test the live plotting. 
plotmon.close()
from importlib import reload
reload(pqm)
plotmon = pqm.PlotMonitor_pyqt('plotmon')